In [10]:
from pymongo import MongoClient
import pandas as pd
client = MongoClient('mongodb://localhost:27017/')
db=client['MongoDB_project']
collection = db['games']
documents = collection.find()
games_df = pd.DataFrame(list(documents))
games_df

,_id,game_id,competition_id,season,round,date,home_club_id,away_club_id,home_club_goals,away_club_goals,...,stadium,attendance,referee,url,home_club_name,away_club_name,aggregate,competition_type,home_club_formation,away_club_formation
0,6613c0f854a57856bc01b85c,2222597,RU1,2012,6. Matchday,2012-08-25,3725,232,2,1,...,Akhmat-Arena,21700.0,Vladislav Bezborodov,https://www.transfermarkt.co.uk/terek-grozny_s...,RFK Akhmat Grozny,FK Spartak Moskva,2:1,domestic_league,NaN,NaN
1,6613c0f854a57856bc01b85d,2222627,RU1,2012,5. Matchday,2012-08-20,2696,4128,0,2,...,Metallurg,11400.0,Sergey Ivanov,https://www.transfermarkt.co.uk/krylya-sovetov...,PFK Krylya Sovetov Samara,Amkar Perm,0:2,domestic_league,NaN,NaN
2,6613c0f854a57856bc01b85e,2222658,RU1,2012,10. Matchday,2012-09-30,2410,121,0,2,...,Arena Khimki,12000.0,Sergey Karasev,https://www.transfermarkt.co.uk/cska-moscow_di...,PFK CSKA Moskva,FK Dinamo Moskva,0:2,domestic_league,NaN,NaN
3,6613c0f854a57856bc01b85f,2222664,RU1,2012,8. Matchday,2012-09-15,932,2698,1,0,...,RZD Arena,11408.0,Sergey Karasev,https://www.transfermarkt.co.uk/lokomotiv-mosc...,"Футбольный клуб ""Локомотив"" Москва",FC Rubin Kazan,1:0,domestic_league,NaN,NaN
4,6613c0f854a57856bc01b860,2222683,RU1,2012,12. Matchday,2012-10-22,2696,12438,0,1,...,Metallurg,7534.0,Timur Arslanbekov,https://www.transfermarkt.co.uk/krylya-sovetov...,PFK Krylya Sovetov Samara,Volga Nizhniy Novgorod (- 2016),0:1,domestic_league,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65211,6613c10054a57856bc02b718,2367292,EL,2013,Group B,2013-11-07,383,419,2,0,...,Philips Stadion,10000.0,Cristian Balaj,https://www.transfermarkt.co.uk/spielbericht/i...,Eindhovense Voetbalvereniging Philips Sport Ve...,NaN,2:0,international_cup,NaN,NaN
65212,6613c10054a57856bc02b719,2629035,NLP,2015,Third Round,2015-10-27,383,16866,6,0,...,Philips Stadion,31000.0,Allard Lindhout,https://www.transfermarkt.co.uk/spielbericht/i...,Eindhovense Voetbalvereniging Philips Sport Ve...,NaN,6:0,domestic_cup,NaN,NaN
65213,6613c10054a57856bc02b71a,3092951,CLQ,2018,Qualifying Round 2nd leg,2018-08-29,383,713,3,0,...,Philips Stadion,34200.0,Anthony Taylor,https://www.transfermarkt.co.uk/spielbericht/i...,Eindhovense Voetbalvereniging Philips Sport Ve...,NaN,3:0,international_cup,NaN,NaN
65214,6613c10054a57856bc02b71b,4164101,CGB,2023,Second Round,2023-08-29,512,1194,6,1,...,bet365 Stadium,9410.0,Michael Salisbury,https://www.transfermarkt.co.uk/spielbericht/i...,Stoke City,NaN,6:1,other,4-2-3-1,4-2-3-1
